In [1]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [2]:
class SomeObject:
    
    def __init__(self):
        self.integer_field = 0
        self.float_field = 0.0
        self.string_field = ""

In [3]:
class EventGet:
    
    def __init__(self, value_type):
        self.kind = 'get'
        self.value_type = value_type

class EventSet:
    
    def __init__(self, value):
        self.kind = 'set'
        self.value_type = type(value)
        self.value = value
    

In [25]:
class NullHandler:
    
    def __init__(self, successor=None):
        self.__successor = successor
        print(f'in init, successor: {self.__successor}')
    
    def handle(self, obj, event):
        print(f'in Null_handle, obj: {obj}, event: {event}')
        if self.__successor:
            print(f'self.__successor: {self.__successor}')
            return self.__successor.handle(obj, event)
            

class IntHandler(NullHandler):
    
    def handle(self, obj, event):
        if event.value_type == int:
            if event.kind == 'get':
                return obj.integer_field
                
            elif event.kind == 'set':
                obj.integer_field = event.value
                
        else:
            print ('in int_handler, go to next')
            return super().handle(obj, event)
    

class FloatHandler(NullHandler):
    
    def handle(self, obj, event):
        print(f'event.value_type: {event.value_type}')
        if event.value_type == float:
            print('event.value_type == float')
            if event.kind == 'get':
                print('event.kind == get')
                print(f'obj.string_field: {obj.float_field}')

                return obj.float_field
                
            elif event.kind == 'set':
                obj.float_field = event.value
                
        else:
            print ('in float_handler, go to next')
            return super().handle(obj, event)
    
    

class StrHandler(NullHandler):
    
    def handle(self, obj, event):
        if event.value_type == str:
            if event.kind == 'get':
                print(f'obj.string_field: {obj.string_field}')
                return obj.string_field
                
            elif event.kind == 'set':
                obj.string_field = event.value
                
        else:
            print ('in string_handler, go to next')
            return super().handle(obj, event)

In [26]:
obj = SomeObject()
obj.integer_field = 42
obj.float_field = 3.14
obj.string_field = "some text"

In [27]:
chain = IntHandler(FloatHandler(StrHandler(NullHandler)))

in init, successor: <class '__main__.NullHandler'>
in init, successor: <__main__.StrHandler object at 0x0000027E8D533C88>
in init, successor: <__main__.FloatHandler object at 0x0000027E8D55E188>


In [28]:
>>> chain.handle(obj, EventGet(int))
# 42

42

In [29]:
chain.handle(obj, EventGet(float))
# 3.14

in int_handler, go to next
in Null_handle, obj: <__main__.SomeObject object at 0x0000027E8D5454C8>, event: <__main__.EventGet object at 0x0000027E8D516688>
self.__successor: <__main__.FloatHandler object at 0x0000027E8D55E188>
event.value_type: <class 'float'>
event.value_type == float
event.kind == get
obj.string_field: 3.14


3.14

In [30]:
>>> chain.handle(obj, EventGet(str))
# 'some text'

in int_handler, go to next
in Null_handle, obj: <__main__.SomeObject object at 0x0000027E8D5454C8>, event: <__main__.EventGet object at 0x0000027E8D56AD88>
self.__successor: <__main__.FloatHandler object at 0x0000027E8D55E188>
event.value_type: <class 'str'>
in float_handler, go to next
in Null_handle, obj: <__main__.SomeObject object at 0x0000027E8D5454C8>, event: <__main__.EventGet object at 0x0000027E8D56AD88>
self.__successor: <__main__.StrHandler object at 0x0000027E8D533C88>
obj.string_field: some text


'some text'

In [31]:
>>> chain.handle(obj, EventSet(100))
>>> chain.handle(obj, EventGet(int))
# 100

100

In [32]:
>>> chain.handle(obj, EventSet(0.5))
>>> chain.handle(obj, EventGet(float))
# 0.5

in int_handler, go to next
in Null_handle, obj: <__main__.SomeObject object at 0x0000027E8D5454C8>, event: <__main__.EventSet object at 0x0000027E8D5642C8>
self.__successor: <__main__.FloatHandler object at 0x0000027E8D55E188>
event.value_type: <class 'float'>
event.value_type == float
in int_handler, go to next
in Null_handle, obj: <__main__.SomeObject object at 0x0000027E8D5454C8>, event: <__main__.EventGet object at 0x0000027E8D564448>
self.__successor: <__main__.FloatHandler object at 0x0000027E8D55E188>
event.value_type: <class 'float'>
event.value_type == float
event.kind == get
obj.string_field: 0.5


0.5

In [33]:
>>> chain.handle(obj, EventSet('new text'))
>>> chain.handle(obj, EventGet(str))
# 'new text'

in int_handler, go to next
in Null_handle, obj: <__main__.SomeObject object at 0x0000027E8D5454C8>, event: <__main__.EventSet object at 0x0000027E8D51EB88>
self.__successor: <__main__.FloatHandler object at 0x0000027E8D55E188>
event.value_type: <class 'str'>
in float_handler, go to next
in Null_handle, obj: <__main__.SomeObject object at 0x0000027E8D5454C8>, event: <__main__.EventSet object at 0x0000027E8D51EB88>
self.__successor: <__main__.StrHandler object at 0x0000027E8D533C88>
in int_handler, go to next
in Null_handle, obj: <__main__.SomeObject object at 0x0000027E8D5454C8>, event: <__main__.EventGet object at 0x0000027E8D51EAC8>
self.__successor: <__main__.FloatHandler object at 0x0000027E8D55E188>
event.value_type: <class 'str'>
in float_handler, go to next
in Null_handle, obj: <__main__.SomeObject object at 0x0000027E8D5454C8>, event: <__main__.EventGet object at 0x0000027E8D51EAC8>
self.__successor: <__main__.StrHandler object at 0x0000027E8D533C88>
obj.string_field: new text


'new text'